In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import random
import json
import zipfile
import io
import math
import time
from collections import deque, defaultdict
from copy import deepcopy
from dataclasses import dataclass
from typing import List, Tuple, Optional

# ============================================================================
# 1. DATA STRUCTURES
# ============================================================================
@dataclass
class Move:
    start: Tuple[int, int]
    end: Tuple[int, int]
    captures: List[Tuple[int, int]]
    promotion: bool = False
    
    def __hash__(self): return hash((self.start, self.end, tuple(self.captures)))
    def __eq__(self, other): return (self.start == other.start and self.end == other.end and self.captures == other.captures)

# ============================================================================
# 2. CHECKERS ENVIRONMENT (Headless)
# ============================================================================
class Checkers:
    def __init__(self):
        self.board_size = 8
        self.reset()
    
    def reset(self):
        self.board = np.zeros((8, 8), dtype=int)
        for row in range(5, 8):
            for col in range(8):
                if (row + col) % 2 == 1: self.board[row, col] = 1
        for row in range(0, 3):
            for col in range(8):
                if (row + col) % 2 == 1: self.board[row, col] = 2
        self.current_player = 1
        self.game_over = False
        self.winner = None
        self.move_history = []
        self.pieces_count = {1: 12, 2: 12}
        return self.get_state()
    
    def get_state(self): return tuple(self.board.flatten())
    
    def copy(self):
        new_game = Checkers()
        new_game.board = self.board.copy()
        new_game.current_player = self.current_player
        new_game.game_over = self.game_over
        new_game.winner = self.winner
        new_game.move_history = self.move_history.copy()
        new_game.pieces_count = self.pieces_count.copy()
        return new_game
    
    def get_piece_moves(self, row, col):
        piece = self.board[row, col]
        if piece == 0 or abs(piece) != self.current_player: return []
        is_king = abs(piece) > 2
        captures = self._get_captures(row, col, is_king)
        if captures: return captures
        return self._get_simple_moves(row, col, is_king)
    
    def _get_simple_moves(self, row, col, is_king):
        moves = []
        piece = self.board[row, col]
        if piece == 1: directions = [(-1, -1), (-1, 1)]
        elif piece == 2: directions = [(1, -1), (1, 1)]
        else: directions = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
        
        for dr, dc in directions:
            nr, nc = row + dr, col + dc
            if 0 <= nr < 8 and 0 <= nc < 8 and self.board[nr, nc] == 0:
                promotion = (piece == 1 and nr == 0) or (piece == 2 and nr == 7)
                moves.append(Move((row, col), (nr, nc), [], promotion))
        return moves
    
    def _get_captures(self, row, col, is_king, captured=None):
        if captured is None: captured = []
        piece = self.board[row, col]
        moves = []
        if piece == 1: directions = [(-1, -1), (-1, 1)]
        elif piece == 2: directions = [(1, -1), (1, 1)]
        else: directions = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
        
        for dr, dc in directions:
            jr, jc = row + dr, col + dc
            lr, lc = row + 2*dr, col + 2*dc
            if not (0 <= lr < 8 and 0 <= lc < 8): continue
            
            enemy = self.board[jr, jc]
            if (enemy != 0 and (enemy % 3) == (3 - self.current_player) and 
                self.board[lr, lc] == 0 and (jr, jc) not in captured):
                
                new_captured = captured + [(jr, jc)]
                orig_board = self.board.copy()
                self.board[lr, lc] = piece; self.board[row, col] = 0
                for cr, cc in new_captured: self.board[cr, cc] = 0
                
                further = self._get_captures(lr, lc, is_king, new_captured)
                self.board = orig_board
                
                if further: moves.extend(further)
                else:
                    start = self.move_history[-1].end if captured else (row, col)
                    promo = ((piece == 1 and lr == 0) or (piece == 2 and lr == 7))
                    moves.append(Move(start, (lr, lc), new_captured, promo))
        return moves
    
    def get_all_valid_moves(self):
        all_moves = []
        has_captures = False
        for r in range(8):
            for c in range(8):
                p = self.board[r, c]
                if p != 0 and abs(p) % 3 == self.current_player:
                    moves = self.get_piece_moves(r, c)
                    if moves and moves[0].captures: has_captures = True
                    all_moves.extend(moves)
        if has_captures: all_moves = [m for m in all_moves if m.captures]
        return all_moves
    
    def make_move(self, move):
        if self.game_over: return self.get_state(), 0, True
        sr, sc = move.start; er, ec = move.end
        piece = self.board[sr, sc]
        self.board[er, ec] = piece; self.board[sr, sc] = 0
        
        points = 0
        for cr, cc in move.captures:
            cap = self.board[cr, cc]; self.board[cr, cc] = 0
            points += 3 if abs(cap) > 2 else 1
            self.pieces_count[3 - self.current_player] -= 1
            
        if move.promotion or (piece == 1 and er == 0) or (piece == 2 and er == 7):
            self.board[er, ec] = piece + 2; points += 2
            
        self.move_history.append(move)
        reward = points
        opponent = 3 - self.current_player
        
        if self.pieces_count[opponent] == 0:
            self.game_over = True; self.winner = self.current_player; reward = 100
        else:
            self.current_player = opponent
            if not self.get_all_valid_moves():
                self.game_over = True; self.winner = 3 - opponent; self.current_player = 3 - opponent; reward = 100
        return self.get_state(), reward, self.game_over
    
    def evaluate_position(self, player):
        if self.winner == player: return 100000
        if self.winner == (3 - player): return -100000
        opp = 3 - player; score = 0
        pst = [[0, 4, 0, 4, 0, 4, 0, 4], [4, 0, 3, 0, 3, 0, 3, 0], [0, 3, 0, 2, 0, 2, 0, 4], [4, 0, 5, 0, 5, 0, 3, 0],
               [0, 3, 0, 5, 0, 5, 0, 4], [4, 0, 2, 0, 2, 0, 3, 0], [0, 3, 0, 3, 0, 3, 0, 4], [4, 0, 4, 0, 4, 0, 4, 0]]
        for r in range(8):
            for c in range(8):
                p = self.board[r, c]
                if p == 0: continue
                is_mine = (abs(p) % 3 == player)
                val = 500 if abs(p) > 2 else 100
                pos_val = pst[r][c] if player == 1 else pst[7-r][c]
                adv = (7 - r) * 3 if player == 1 else r * 3
                if is_mine: score += val + (pos_val * 5) + adv
                else: score -= val + (pos_val * 5) + adv
        return score

# ============================================================================
# 3. MCTS & AGENT
# ============================================================================
class MCTSNode:
    def __init__(self, game, parent=None, move=None, prior=1.0):
        self.game = game; self.parent = parent; self.move = move; self.prior = prior
        self.children = {}; self.visit_count = 0; self.value_sum = 0.0; self.is_expanded = False
    def value(self): return self.value_sum / self.visit_count if self.visit_count > 0 else 0
    def ucb(self, pv, c=1.5):
        q = self.value() if self.visit_count > 0 else 0
        return q + c * self.prior * math.sqrt(pv) / (1 + self.visit_count)
    def select(self, c=1.5): return max(self.children.values(), key=lambda c_node: c_node.ucb(self.visit_count, c))
    def expand(self, game, priors):
        moves = game.get_all_valid_moves(); tp = sum(priors.values()) or len(moves)
        for m in moves:
            p = priors.get(m, 1.0) / tp
            cg = game.copy(); cg.make_move(m)
            self.children[m] = MCTSNode(cg, self, m, p)
        self.is_expanded = True
    def backup(self, v):
        self.visit_count += 1; self.value_sum += v
        if self.parent: self.parent.backup(-v)

class Agent:
    def __init__(self, pid, lr=0.3, gamma=0.99):
        self.player_id = pid; self.lr = lr; self.gamma = gamma
        self.epsilon = 1.0; self.epsilon_decay = 0.96; self.epsilon_min = 0.01
        self.mcts_simulations = 250; self.minimax_depth = 4
        self.policy_table = defaultdict(lambda: defaultdict(float))
        self.wins = 0; self.losses = 0; self.draws = 0

    def get_priors(self, game):
        state = game.get_state(); moves = game.get_all_valid_moves(); priors = {}
        for m in moves:
            if state in self.policy_table and m in self.policy_table[state]: priors[m] = self.policy_table[state][m]
            else:
                p = 1.0 + (len(m.captures) * 2) + (1.0 if m.promotion else 0)
                if 2 <= m.end[0] <= 5 and 2 <= m.end[1] <= 5: p += 0.5
                priors[m] = p
        return priors

    def mcts(self, game, sims):
        root = MCTSNode(game.copy())
        for _ in range(sims):
            node = root; sg = game.copy()
            while node.is_expanded and node.children:
                node = node.select(); sg.make_move(node.move)
            if not sg.game_over: node.expand(sg, self.get_priors(sg))
            val = self._eval_leaf(sg); node.backup(val)
        return root

    def _eval_leaf(self, game):
        if game.game_over: return 1.0 if game.winner == self.player_id else (-1.0 if game.winner else 0.0)
        return np.tanh(self._minimax(game, self.minimax_depth, -float('inf'), float('inf'), True) / 500)

    def _minimax(self, g, d, a, b, maxing):
        if d == 0 or g.game_over: return g.evaluate_position(self.player_id)
        moves = g.get_all_valid_moves()[:4] # Limit search width
        if maxing:
            me = -float('inf')
            for m in moves:
                cg = g.copy(); cg.make_move(m)
                me = max(me, self._minimax(cg, d-1, a, b, False)); a = max(a, me)
                if b <= a: break
            return me
        else:
            me = float('inf')
            for m in moves:
                cg = g.copy(); cg.make_move(m)
                me = min(me, self._minimax(cg, d-1, a, b, True)); b = min(b, me)
                if b <= a: break
            return me

    def choose_action(self, game, training=True):
        moves = game.get_all_valid_moves()
        if not moves: return None
        if training and random.random() < self.epsilon: return random.choice(moves)
        root = self.mcts(game, self.mcts_simulations)
        if not root.children: return random.choice(moves)
        best = max(root.children.items(), key=lambda x: x[1].visit_count)[0]
        
        # Policy Learning
        s = game.get_state(); tv = sum(c.visit_count for c in root.children.values())
        for m, c in root.children.items(): self.policy_table[s][m] = c.visit_count / tv
        return best

    def update(self, hist, res):
        for s, m, p in hist:
            if p != self.player_id: continue
            r = 1.0 if res == self.player_id else (0.0 if res == 0 else -1.0)
            curr = self.policy_table[s][m]
            self.policy_table[s][m] = curr + self.lr * (r - curr)
            
    def decay_epsilon(self): self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    def reset_stats(self): self.wins=0; self.losses=0; self.draws=0

def play_game(env, a1, a2, training=True):
    env.reset(); hist = []; agents = {1: a1, 2: a2}; mc = 0
    while not env.game_over and mc < 200:
        cp = env.current_player; ag = agents[cp]
        s = env.get_state(); m = ag.choose_action(env, training)
        if m is None: break
        hist.append((s, m, cp)); env.make_move(m); mc += 1
    
    if env.winner == 1:
        a1.wins+=1; a2.losses+=1
        if training: a1.update(hist, 1); a2.update(hist, 1)
    elif env.winner == 2:
        a2.wins+=1; a1.losses+=1
        if training: a1.update(hist, 2); a2.update(hist, 2)
    else:
        a1.draws+=1; a2.draws+=1
        if training: a1.update(hist, 0); a2.update(hist, 0)
    return env.winner

In [5]:
# ============================================================================
# 1. SERIALIZATION (Matches Streamlit deserialize_move EXACTLY)
# ============================================================================
def serialize_move(move):
    """Convert Move object to JSON-dict for Streamlit compatibility"""
    return {
        "s": [int(x) for x in move.start],     # 's' for start
        "e": [int(x) for x in move.end],       # 'e' for end
        "c": [[int(x) for x in c] for c in move.captures], # 'c' for captures
        "p": bool(move.promotion)              # 'p' for promotion
    }

def save_kaggle_brain(agent1, agent2, config, filename="checkers.zip"):
    # Helper to serialize an agent's policy table
    def serialize_agent(agent, role):
        clean_policy = {}
        for state, moves in agent.policy_table.items():
            try:
                # 1. Convert state tuple (ints) to string key
                clean_state = tuple(int(x) for x in state)
                state_str = str(clean_state)
                
                clean_policy[state_str] = {}
                for move, value in moves.items():
                    # 2. Serialize Move object using the helper above
                    move_json_str = json.dumps(serialize_move(move))
                    clean_policy[state_str][move_json_str] = float(value)
            except Exception: continue
            
        return {
            "metadata": {"role": role, "version": "KAGGLE_V1"},
            "policy_table": clean_policy,
            "epsilon": float(agent.epsilon),
            "wins": int(agent.wins),
            "losses": int(agent.losses),
            "draws": int(agent.draws),
            "mcts_sims": int(agent.mcts_simulations)
        }

    a1_data = serialize_agent(agent1, "Red")
    a2_data = serialize_agent(agent2, "White")
    
    print(f"💾 Saving: Red Policies={len(a1_data['policy_table'])}, White Policies={len(a2_data['policy_table'])}")
    
    with zipfile.ZipFile(filename, "w", zipfile.ZIP_DEFLATED) as zf:
        zf.writestr("agent1.json", json.dumps(a1_data, indent=2))
        zf.writestr("agent2.json", json.dumps(a2_data, indent=2))
        zf.writestr("config.json", json.dumps(config, indent=2))
    print(f"✅ Save Complete! Download '{filename}'")

# ============================================================================
# 2. TRAINING RUNNER
# ============================================================================
def run_checkers_training():
    EPISODES = 50  # Checkers is slow; 500 is a good start. Try 2000 if patient.
    
    env = Checkers()
    agent1 = Agent(1, lr=0.3, gamma=0.99)
    agent2 = Agent(2, lr=0.3, gamma=0.99)
    
    # Brain Power
    agent1.mcts_simulations = 100
    agent1.minimax_depth = 5
    agent2.mcts_simulations = 100
    agent2.minimax_depth = 5
    
    history = {
        'agent1_wins': [], 'agent2_wins': [], 'draws': [],
        'agent1_epsilon': [], 'agent2_epsilon': [],
        'agent1_policies': [], 'agent2_policies': [],
        'episode': []
    }
    
    print(f"🚀 Starting Checkers Training ({EPISODES} Episodes)...")
    start_time = time.time()
    
    for ep in range(1, EPISODES + 1):
        play_game(env, agent1, agent2, training=True)
        
        agent1.decay_epsilon()
        agent2.decay_epsilon()
        
        if ep % 10 == 0:
            history['agent1_wins'].append(agent1.wins)
            history['agent2_wins'].append(agent2.wins)
            history['draws'].append(agent1.draws)
            history['agent1_epsilon'].append(agent1.epsilon)
            history['agent2_epsilon'].append(agent2.epsilon)
            history['agent1_policies'].append(len(agent1.policy_table))
            history['agent2_policies'].append(len(agent2.policy_table))
            history['episode'].append(ep)
            
        if ep % 1 == 0:
            elapsed = time.time() - start_time
            print(f"Ep {ep}/{EPISODES} | Red Wins: {agent1.wins} | Wht Wins: {agent2.wins} | {elapsed:.1f}s")

    config = {
        "lr1": agent1.lr, "gamma1": agent1.gamma,
        "lr2": agent2.lr, "gamma2": agent2.gamma,
        "training_history": history
    }
    
    print("\n🏆 Training Finished!")
    save_kaggle_brain(agent1, agent2, config)

if __name__ == "__main__":
    run_checkers_training()

🚀 Starting Checkers Training (50 Episodes)...
Ep 1/50 | Red Wins: 1 | Wht Wins: 0 | 0.0s
Ep 2/50 | Red Wins: 1 | Wht Wins: 1 | 2.6s
Ep 3/50 | Red Wins: 1 | Wht Wins: 2 | 4.8s
Ep 4/50 | Red Wins: 1 | Wht Wins: 3 | 13.5s
Ep 5/50 | Red Wins: 1 | Wht Wins: 4 | 24.6s
Ep 6/50 | Red Wins: 1 | Wht Wins: 5 | 39.9s
Ep 7/50 | Red Wins: 1 | Wht Wins: 6 | 47.6s
Ep 8/50 | Red Wins: 2 | Wht Wins: 6 | 62.0s
Ep 9/50 | Red Wins: 2 | Wht Wins: 7 | 76.4s
Ep 10/50 | Red Wins: 3 | Wht Wins: 7 | 95.6s
Ep 11/50 | Red Wins: 3 | Wht Wins: 8 | 121.5s
Ep 12/50 | Red Wins: 4 | Wht Wins: 8 | 138.7s
Ep 13/50 | Red Wins: 5 | Wht Wins: 8 | 162.2s
Ep 14/50 | Red Wins: 6 | Wht Wins: 8 | 181.4s
Ep 15/50 | Red Wins: 6 | Wht Wins: 9 | 207.3s
Ep 16/50 | Red Wins: 7 | Wht Wins: 9 | 236.8s
Ep 17/50 | Red Wins: 8 | Wht Wins: 9 | 269.7s
Ep 18/50 | Red Wins: 8 | Wht Wins: 10 | 296.2s
Ep 19/50 | Red Wins: 8 | Wht Wins: 11 | 311.4s
Ep 20/50 | Red Wins: 8 | Wht Wins: 12 | 346.9s
Ep 21/50 | Red Wins: 9 | Wht Wins: 12 | 376.7s
Ep 22/